In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup as bs
driver=webdriver.Chrome("D:\Driver\chromedriver.exe")

In [4]:
driver.get("http://www.eslitecorp.com/Site/Site_List.aspx?a=TW&l=b&sc=TW")

In [5]:
soup=bs(driver.page_source,'lxml')

In [6]:
driver.quit()

In [7]:
datas=[]
for li in soup.select("div.main_list > ul > li"):
    data={}
    data['name']=li.select_one("a > div > h6").text
    data['address']=li.select_one("a > div > p").text
    datas.append(data)

In [8]:
datas

[{'address': '花蓮市和平路581號2F〈遠東百貨2F〉', 'name': '誠品生活花蓮遠百店'},
 {'address': '台北市大同區南京西路16號B1之中山地下街B1-B48號店鋪（不含B24、B26號）', 'name': '誠品R79'},
 {'address': '高雄市前鎮區三多三路217號12F (高雄SOGO百貨12F)', 'name': '高雄SOGO店'},
 {'address': '臺中市西屯區臺灣大道三段251號', 'name': '台中大遠百店'},
 {'address': '新北市林口區文化三路一段356號1F', 'name': '林口店'},
 {'address': '臺北市大安區忠孝東路四段45號12F', 'name': '忠孝SOGO店'},
 {'address': '高雄市鹽埕區大勇路3號 (駁二藝術特區C4倉庫)', 'name': '高雄駁二店'},
 {'address': '台南市東區中華東路一段366號2樓', 'name': '南紡店'},
 {'address': '臺北市敦化南路一段245號', 'name': '敦南店'},
 {'address': '桃園市蘆竹區南崁路一段112號6F（台茂購物中心6F）', 'name': '桃園台茂店'},
 {'address': '臺北市信義區松高路11號', 'name': '信義店'},
 {'address': '臺北市信義區菸廠路88號', 'name': '松菸店'},
 {'address': '臺北市忠孝西路一段47號B1（臺北車站K區地下街）', 'name': '站前店'},
 {'address': '臺北市萬華區武昌街2段77號', 'name': '武昌店'},
 {'address': '臺北市新生南路三段98號B1~3F', 'name': '台大店'},
 {'address': '臺北市峨嵋街52號', 'name': '西門店'},
 {'address': '臺北市忠孝西路一段49號B1（B、C、D室）', 'name': '台北車站捷運店'},
 {'address': '新北市板橋區中山路一段46號', 'name': '板橋店'},
 {'address': '臺北市康寧路三段72號1F/

In [9]:
import geocoder
import time
for j in range(2):
    for i in datas:
        if 'lat' not in i:
            newAdd=i['address'].split("-")[0].split("~")[0].split("、")[0].split("號")[0]
            i['lat'],i['lng']=geocoder.arcgis(newAdd).latlng
            print(newAdd,i['lat'],i['lng'])
            time.sleep(1)
        
for i in datas:
    if 'lat' not in i:
        newAdd=i['address'].split("-")[0].split("~")[0].split("、")[0].split("號")[0]
        i['lat'],i['lng']=geocoder.google(newAdd).latlng
        print(newAdd,i['lat'],i['lng'])
        time.sleep(1.5)

花蓮市和平路581 23.97814094226446 121.60018533943106
台北市大同區南京西路16 25.052580776559633 121.52034395545043
高雄市前鎮區三多三路217 22.614333738530924 120.30582472107447
臺中市西屯區臺灣大道三段251 24.165051425443142 120.64482359559139
新北市林口區文化三路一段356 25.071086712476095 121.36325690524534
臺北市大安區忠孝東路四段45 25.04171452596792 121.54487883527611
高雄市鹽埕區大勇路3 22.62017323731884 120.28193221784313
台南市東區中華東路一段366 22.99122042936694 120.23378986023442
臺北市敦化南路一段245 25.039116000000007 121.54910632474453
桃園市蘆竹區南崁路一段112 25.052939808528816 121.28770344993919
臺北市信義區松高路11 25.039126486005973 121.56585708090483
臺北市信義區菸廠路88 25.044954623348286 121.56145717842944
臺北市忠孝西路一段47 25.046326727739537 121.51739369401001
臺北市萬華區武昌街2段77 25.04507187316061 121.50570295115125
臺北市新生南路三段98 25.01697356924808 121.53318365055694
臺北市峨嵋街52 25.037370000000067 121.56355000000008
臺北市忠孝西路一段49 25.046651452653048 121.51622848835729
新北市板橋區中山路一段46 25.0089570843673 121.46073201416695
臺北市康寧路三段72 25.0701582470137 121.61145134831789
臺北市文林路338 25.092704000836722 121.525740308

In [24]:
from pymongo import MongoClient,UpdateOne
client=MongoClient("192.168.1.113",27017,username='root',password='850605')

In [25]:
colle=client.rawData.eslite

In [26]:
ids=[i['name'] for i in datas]

In [27]:
operation=[UpdateOne({"id":idn},{"$set":data},upsert=True) for idn,data in zip(ids,datas)]

In [28]:
colle.bulk_write(operation)

In [29]:
client.close()

In [32]:
datas[:8]

[{'address': '花蓮市和平路581號2F〈遠東百貨2F〉',
  'lat': 23.97814094226446,
  'lng': 121.60018533943106,
  'name': '誠品生活花蓮遠百店'},
 {'address': '台北市大同區南京西路16號B1之中山地下街B1-B48號店鋪（不含B24、B26號）',
  'lat': 25.052580776559633,
  'lng': 121.52034395545043,
  'name': '誠品R79'},
 {'address': '高雄市前鎮區三多三路217號12F (高雄SOGO百貨12F)',
  'lat': 22.614333738530924,
  'lng': 120.30582472107447,
  'name': '高雄SOGO店'},
 {'address': '臺中市西屯區臺灣大道三段251號',
  'lat': 24.165051425443142,
  'lng': 120.64482359559139,
  'name': '台中大遠百店'},
 {'address': '新北市林口區文化三路一段356號1F',
  'lat': 25.071086712476095,
  'lng': 121.36325690524534,
  'name': '林口店'},
 {'address': '臺北市大安區忠孝東路四段45號12F',
  'lat': 25.04171452596792,
  'lng': 121.54487883527611,
  'name': '忠孝SOGO店'},
 {'address': '高雄市鹽埕區大勇路3號 (駁二藝術特區C4倉庫)',
  'lat': 22.62017323731884,
  'lng': 120.28193221784313,
  'name': '高雄駁二店'},
 {'address': '台南市東區中華東路一段366號2樓',
  'lat': 22.99122042936694,
  'lng': 120.23378986023442,
  'name': '南紡店'}]